# Get TCBY locations

#### Load Python tools and Jupyter config

In [1]:
%load_ext lab_black

In [2]:
import json
import requests
import warnings
import pandas as pd
import altair as alt
from time import sleep
import geopandas as gpd
from random import randint
from vega_datasets import data
from tqdm.notebook import tqdm, trange

In [3]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = None
warnings.simplefilter(action="ignore")

In [4]:
place = "TCBY"

## Get data

#### Function for accessing the state locations data

In [5]:
def fetch_state_locations(state):
    url = f"https://www.tcby.com/api/geo/{state}/"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return []

#### All the states

In [6]:
states = [
    "AL",
    "AK",
    "AZ",
    "AR",
    "CA",
    "CO",
    "CT",
    "DE",
    "FL",
    "GA",
    "HI",
    "ID",
    "IL",
    "IN",
    "IA",
    "KS",
    "KY",
    "LA",
    "ME",
    "MD",
    "MA",
    "MI",
    "MN",
    "MS",
    "MO",
    "MT",
    "NE",
    "NV",
    "NH",
    "NJ",
    "NM",
    "NY",
    "NC",
    "ND",
    "OH",
    "OK",
    "OR",
    "PA",
    "RI",
    "SC",
    "SD",
    "TN",
    "TX",
    "UT",
    "VT",
    "VA",
    "WA",
    "WV",
    "WI",
    "WY",
]

#### Loop through out list of states, grab returned lists of locations for each

In [7]:
all_locations = []

for state in tqdm(states):
    state_locations = fetch_state_locations(state)
    all_locations.extend(state_locations)

  0%|          | 0/50 [00:00<?, ?it/s]

#### Put that into a dataframe

In [8]:
src = pd.DataFrame(all_locations)

#### How many did we get?

In [9]:
len(src)

135

#### Just the columns we need

In [10]:
df = src[
    [
        "store_number",
        "name",
        "address",
        "city",
        "zip",
        "state",
        "website",
        "phone",
        "latitude",
        "longitude",
    ]
].copy()

In [11]:
df.head()

,store_number,name,address,city,zip,state,website,phone,latitude,longitude
0,9017905,Essex Square Market Place,2 McFarland Blvd,Northport,35476,AL,https://store.tcby.com/home/essexsquaremarketplace,(205) 758-6855,33.2361582,-87.5523731
1,9017906,Meadowbrook,2304 McFarland Blvd E,Tuscaloosa,35404,AL,https://store.tcby.com/home/meadowbrook,(205) 349-4661,33.1889321,-87.526194
2,9291601,Corner Village,300 N Dean Rd,Auburn,36830,AL,https://store.tcby.com/home/auburntcby,(334) 826-8828,32.6100321,-85.4648807
3,9237207,Gulf Shores,1301 Gulf Shores Pkwy,Gulf Shores,36542,AL,https://store.tcby.com/home/gulfshoresal,(251) 968-8229,30.2629312,-87.688844
4,9448510,Merchants Walk,1919 28th Ave S,Birmingham,35209,AL,https://store.tcby.com/home/merchantswalk,(205) 870-8229,33.480864,-86.7882114


---

## Geography

#### Make it a geodataframe

In [12]:
df_geo = df.copy()

In [13]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude)
)

In [14]:
locations_gdf = gdf.set_crs("EPSG:4326").copy()

---

## Maps

#### US states background

In [15]:
background = (
    alt.Chart(alt.topo_feature(data.us_10m.url, feature="states"))
    .mark_geoshape(fill="#e9e9e9", stroke="white")
    .properties(width=800, height=500, title=f"{place} locations")
    .project("albersUsa")
)

#### Location points map

In [16]:
points = (
    alt.Chart(gdf)
    .mark_circle(size=10, color="#ec008b")
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
    )
)

point_map = background + points
point_map.configure_view(stroke=None)

alt.LayerChart(...)

#### Location proportional symbols map

In [17]:
symbols = (
    alt.Chart(gdf)
    .transform_aggregate(
        latitude="mean(latitude)",
        longitude="mean(longitude)",
        count="count()",
        groupby=["state"],
    )
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("count:Q", title="Count by state"),
        color=alt.value("#ec008b"),
        tooltip=["state:N", "count:Q"],
    )
    .properties(title=f"Number of {place} in US, by average lon/lat of locations")
)

symbol_map = background + symbols
symbol_map.configure_view(stroke=None)

alt.LayerChart(...)

---

## Exports

#### JSON

In [18]:
df.to_json(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.json",
    indent=4,
    orient="records",
)

#### CSV

In [19]:
df.to_csv(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.csv", index=False
)

#### GeoJSON

In [20]:
locations_gdf.to_file(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.geojson",
    driver="GeoJSON",
)